In [25]:
import numpy as np
import pandas as pd

In [26]:
pieces = {
    'L5' : [[1,1,1,1],[0,0,0,1]],
    'Y' : [[1,1,1,1],[0,1,0,0]],
    'N' : [[1,1,1,1],[0,0,1,1]],
    'U' : [[1,1],[1,0],[1,1]],
    'V3' : [[1,0],[1,1]],
    'V5' : [[1,0,0],[1,0,0],[1,1,1]],
    'Z5' : [[1,1,0],[0,1,0],[0,1,1]],
    'X' : [[0,1,0],[1,1,1],[0,1,0]],
    'T5' : [[1,1,1],[0,1,0],[0,1,0]],
    'W' : [[1,0,0],[1,1,0],[0,1,1]],
    'P' : [[1,1],[1,1],[1,0]],
    'F' : [[0,1,1],[1,1,0],[0,1,0]],
    'O4': [[1,1],[1,1]],
    'L4' : [[1,1,1],[0,0,1]],
    'T4' : [[1,1,1],[0,1,0]],
    'Z4' : [[1,1,0],[0,1,1]],
    'I2' : [1,1],
    'I3' : [1,1,1],
    'I4' : [1,1,1,1],
    'I5' : [1,1,1,1,1]
    }

In [27]:
class Blok:
    def __init__(self, blockName):
        self.name = blockName
        self.map = pieces[blockName]

    def rotate(self):
        self.map = np.transpose(self.map)

In [28]:
class Point:    
    def __init__(self, x, y):
        self.x = _x
        self.y = _y

In [29]:
class Player:
    def __init__(self, playerName):
        self.name = playerName
        peaceList = pieces.keys()
        self.pieces = peaceList
        
#     def zen():
#         peaceList = pieces.keys()
#         return peaceList

In [41]:
class Board:
    def __init__(self, **kwargs):
        self.boardState = kwargs.get('BoardState', pd.DataFrame(np.zeros((20,20))))
        self.boardState.index = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t']

        
        
game1 = Board()
game1.boardState

# board = pd.DataFrame(np.zeros((20,20)), index = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t'])
# board

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
d,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
g,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
h,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
i,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
j,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
